In [148]:
!pip install google-cloud-storage google-cloud-bigquery


Defaulting to user installation because normal site-packages is not writeable


In [149]:
from google.cloud import storage, bigquery
import os

---
# Étape 1 : Configuration des ressources GCP

In [150]:
# Configuration de base
PROJECT_ID = "isi-group-m2-dsia"
BUCKET_NAME = "m2dsia-ndao-ibrahima-data"
DATASET_NAME = 'dataset_ndao_ibrahima'
TABLE_NAME = 'auchan_sales'


In [151]:
client = storage.Client(project=PROJECT_ID)
bucket = client.get_bucket(BUCKET_NAME)

In [152]:
bucket

<Bucket: m2dsia-ndao-ibrahima-data>

In [153]:
def create_folder_bucket(bucket_name):
    # Define folder structure
    folders = ["input/", "clean/", "error/", "done/"]
    for folder in folders:
        blob = bucket.blob(folder)
        blob.upload_from_string('')  # Create an empty folder
        print(f"Folder {folder} created in bucket {bucket_name}.")

In [154]:
create_folder_bucket(BUCKET_NAME)

Folder input/ created in bucket m2dsia-ndao-ibrahima-data.
Folder clean/ created in bucket m2dsia-ndao-ibrahima-data.
Folder error/ created in bucket m2dsia-ndao-ibrahima-data.
Folder done/ created in bucket m2dsia-ndao-ibrahima-data.


In [155]:
blobs = client.list_blobs(BUCKET_NAME)
for blob in blobs:
    print(blob.name)

clean/
done/
error/
input/
tp/
tp/Copie de Copie de auchan_sales_data.csv
tp/Copie de auchan_sales_data.csv
tp/auchan_sales_data.csv


In [156]:
def create_bigquery_table(project_id, dataset_name, table_name):
    """Create a BigQuery dataset and table."""
    client = bigquery.Client()

    # Create dataset
    dataset_id = f"{project_id}.{dataset_name}"
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "africa-south1"  # Adjust region as needed
    dataset = client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {dataset_id} created or already exists.")

    # Define table schema
    schema = [
        bigquery.SchemaField("transaction_id", "INT64", mode="REQUIRED", description="Identifiant unique"),
        bigquery.SchemaField("product_name", "STRING", mode="REQUIRED", description="Nom du produit"),
        bigquery.SchemaField("category", "STRING", mode="REQUIRED", description="Catégorie du produit"),
        bigquery.SchemaField("price", "FLOAT64", mode="REQUIRED", description="Prix unitaire"),
        bigquery.SchemaField("quantity", "INT64", mode="REQUIRED", description="Quantité achetée"),
        bigquery.SchemaField("date", "DATE", mode="REQUIRED", description="Date de la transaction"),
        bigquery.SchemaField("customer_name", "STRING", mode="NULLABLE", description="Nom du client"),
        bigquery.SchemaField("customer_email", "STRING", mode="NULLABLE", description="E-mail du client"),
    ]

    table_id = f"{dataset_id}.{table_name}"
    table = bigquery.Table(table_id, schema=schema)

    # Set partitioning and clustering
    table.time_partitioning = bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="date"  # Specify the partitioning field
    )
    table.clustering_fields = ["category", "product_name"]

    # Create the table
    table = client.create_table(table, exists_ok=True)
    print(f"Table {table_id} created or already exists.")

In [157]:
create_bigquery_table(PROJECT_ID, DATASET_NAME, TABLE_NAME)

Dataset isi-group-m2-dsia.dataset_ndao_ibrahima created or already exists.
Table isi-group-m2-dsia.dataset_ndao_ibrahima.auchan_sales created or already exists.


---

In [158]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [159]:
import pandas as pd
import io

In [160]:
def download_file_from_gcs(bucket_name, file_path):
    """Download a file from Google Cloud Storage."""
    # Télécharger le fichier comme texte
    data = blob.download_as_text()
    print(f"File {file_path} downloaded successfully from bucket {bucket_name}.")
    
    # Charger le contenu dans un DataFrame
    dataframe = pd.read_csv(io.StringIO(data))
    return dataframe

In [161]:
file_path = "tp/auchan_sales_data.csv"

In [162]:
dataframe = download_file_from_gcs(BUCKET_NAME, file_path)

File tp/auchan_sales_data.csv downloaded successfully from bucket m2dsia-ndao-ibrahima-data.


In [163]:
dataframe.head()

,date,category,product,qty,price,revenue
0,25/01/2025,Beverages,Cola 2L,65,1.8,117.0
1,18/01/2025,Hygiene,Soap Bar,159,1.0,159.0
2,02/01/2025,Beverages,Mineral Water 6x1.5L,214,3.0,642.0
3,07/01/2025,Hygiene,Soap Bar,175,1.0,175.0
4,04/01/2025,Beverages,Cola 2L,75,1.8,135.0


In [164]:
def upload_to_bigquery(dataframe, project_id, dataset_name, table_name):
    """Upload a Pandas DataFrame to a BigQuery table."""
    if dataframe is None or not isinstance(dataframe, pd.DataFrame):
        raise ValueError("Le DataFrame fourni est invalide.")
    if dataframe.empty:
        raise ValueError("Le DataFrame est vide.")

    # Convertir les types si nécessaire
    dataframe['date'] = pd.to_datetime(dataframe['date'], errors='coerce')
    dataframe['qty'] = pd.to_numeric(dataframe['qty'], errors='coerce')
    dataframe['price'] = pd.to_numeric(dataframe['price'], errors='coerce')
    dataframe['revenue'] = pd.to_numeric(dataframe['revenue'], errors='coerce')

    print("Aperçu du DataFrame avant chargement :")
    print(dataframe.info())
    print(dataframe.head())

    client = bigquery.Client()
    table_id = f"{project_id}.{dataset_name}.{table_name}"

    # Configuration du job
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
        source_format=bigquery.SourceFormat.CSV,
        autodetect=True,
    )

    # Chargement dans BigQuery
    try:
        job = client.load_table_from_dataframe(dataframe, table_id, job_config=job_config)
        job.result()  # Attend la fin du job
        print(f"Les données ont été chargées avec succès dans {table_id}.")
    except Exception as e:
        print(f"Erreur lors du chargement des données dans BigQuery : {e}")


In [166]:
upload_to_bigquery(dataframe, PROJECT_ID, DATASET_NAME, TABLE_NAME)

C:\Windows\Temp\ipykernel_6968\2973799868.py:9: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dataframe['date'] = pd.to_datetime(dataframe['date'], errors='coerce')


Aperçu du DataFrame avant chargement :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      1000 non-null   datetime64[ns]
 1   category  1000 non-null   object        
 2   product   1000 non-null   object        
 3   qty       1000 non-null   int64         
 4   price     1000 non-null   float64       
 5   revenue   1000 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 47.0+ KB
None
        date   category               product  qty  price  revenue
0 2025-01-25  Beverages               Cola 2L   65    1.8    117.0
1 2025-01-18    Hygiene              Soap Bar  159    1.0    159.0
2 2025-01-02  Beverages  Mineral Water 6x1.5L  214    3.0    642.0
3 2025-01-07    Hygiene              Soap Bar  175    1.0    175.0
4 2025-01-04  Beverages               Cola 2L   75    1.8    135.0